In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import csv
import gc
import tensorflow as tf

from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from IPython.display import display, Image

from os import listdir
from os.path import isfile, join
import random
from PIL import Image

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:
mypath = "images_training_rev1/"
newPath = "training_small3/"
files = listdir(mypath)
newFiles = listdir(newPath)

for id in range(len(files)):
    path = mypath + str(files[id])
    try:
        img = Image.open(path).convert('L');  #greyscale
        half_the_width = img.size[0] / 2
        half_the_height = img.size[1] / 2
        for i in range(0,4):
            degree = i*45;
            img2 = img.rotate(degree) 
            img3 = img2.crop(
                (
                    half_the_width - 100,
                    half_the_height - 100,
                    half_the_width + 100,
                    half_the_height + 100
                )
            )     
            img4 = img3.resize([80,80]) 
            file_name = str(files[id]);
            img4.save(newPath + file_name[0] + "/" + str(i) + "_" + file_name) #salvează imaginea 
        if (id%500 == 0):
            gc.collect()
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')


In [3]:
mypath = "images_training_rev1/"
newPath = "training_80px/"
files = listdir(mypath)
print(len(files))
newFiles = listdir(newPath)
print(len(newFiles))

61578
18


In [3]:
v = np.empty((1000000, 0)).tolist()
with open('training_solutions_rev1.csv', 'rt') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    first = True;
    for row in spamreader:
        if(not(first)):
            v[int(row[0])] = list(map(float, row[1:len(row)]))
        else:
            first = False


In [6]:
image_size = 80  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
train_folders = ['.\\training_80px\\1', '.\\training_80px\\2', '.\\training_80px\\3', '.\\training_80px\\4', '.\\training_80px\\5', '.\\training_80px\\6', '.\\training_80px\\7', '.\\training_80px\\8', '.\\training_80px\\9']

def load_images(folder, min_num_images):
  """Load the data for a single  folder."""

  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)

  labels = np.ndarray(shape=(len(image_files),37))
  ids = np.ndarray(shape=(len(image_files)))
  print(folder)
  num_images = 0

  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      idstr = str(image).replace(".jpg","").replace("0_","").replace("1_","").replace("2_","").replace("3_","")
      id = int(idstr)
      ids[num_images] = id
      labels[num_images] = v[id]
      if (num_images == 500): 
        print(labels[num_images])
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset, labels, ids
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    print(folder)
    
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      #override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset, labels, ids = load_images(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          save = {
            'dataset': dataset,
            'labels': labels,
            'ids': ids,
            }
          pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names
  

train_datasets = maybe_pickle(train_folders, 6000)

.\training_80px\1
Pickling .\training_80px\1.pickle.
.\training_80px\1
[ 0.605956    0.392745    0.001299    0.          0.392745    0.12399942
  0.26874559  0.14848864  0.24425636  0.          0.          0.24732134
  0.14542366  0.581634    0.418366    0.10828737  0.49766863  0.          0.
  0.          0.          0.0343199   0.21641496  0.33089914  0.          0.
  0.          0.          0.          0.14848864  0.          0.
  0.14848864  0.          0.          0.          0.        ]
Full dataset tensor: (27044, 80, 80)
Mean: -0.406139
Standard deviation: 0.127839
.\training_80px\2
Pickling .\training_80px\2.pickle.
.\training_80px\2
[ 0.321804    0.678196    0.          0.678196    0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.051974
  0.948026    0.          0.035876    0.285928    0.051974    0.          0.
  0.          0.          0.          0.          0.35133537  0.
  0.32686063  0.          0.          0.       

In [ ]:
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    assert len(a) == len(c)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]

def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray((nb_rows, 37), dtype=np.float32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0, test_size=0 ):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  test_dataset, test_labels = make_arrays(test_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
  ttsize_per_class = test_size // num_classes
  start_v, start_t, start_tt = 0, 0, 0
  end_v, end_t, end_tt = vsize_per_class, tsize_per_class, ttsize_per_class
  end_l = vsize_per_class + tsize_per_class + ttsize_per_class

  for fd, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        save = pickle.load(f)  # unpickle
        dataset = save['dataset']
        labels = save['labels']
        ids = save['ids']
        # shuffle the images to have random validation and training set
        unison_shuffled_copies(dataset, labels, ids)
        
        if valid_dataset is not None:
          valid_letter = dataset[:vsize_per_class, :, :]
          print(len(valid_letter))
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v,:] = labels[:vsize_per_class, :]
          start_v += vsize_per_class
          end_v += vsize_per_class
        
        if test_dataset is not None:
          test_letter = dataset[vsize_per_class:vsize_per_class+ttsize_per_class, :, :]
          print(len(test_letter))
          test_dataset[start_tt:end_tt, :, :] = test_letter
          test_labels[start_tt:end_tt] = labels[vsize_per_class:vsize_per_class+ttsize_per_class, :]
          start_tt += ttsize_per_class
          end_tt += ttsize_per_class
        
        train_letter = dataset[vsize_per_class+ttsize_per_class:end_l, :, :]

        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = labels[vsize_per_class+ttsize_per_class:end_l, :]
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return train_dataset, train_labels, valid_dataset, valid_labels, test_dataset, test_labels 
            
            
train_size = 200000
valid_size = 20000
test_size = 20000

train_dataset, train_labels, valid_dataset, valid_labels, test_dataset, test_labels = merge_datasets(
  train_datasets, train_size, valid_size, test_size)


print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

In [22]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

In [26]:
data_root = '.'
pickle_file = os.path.join(data_root, 'galaxies.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise
 

In [27]:
import time

def check_overlaps(images1, images2):
    start = time.clock()
    hash1 = set([hash(image1.tobytes()) for image1 in images1])
    hash2 = set([hash(image2.tobytes()) for image2 in images2])
    all_overlaps = set.intersection(hash1, hash2)
    return all_overlaps, time.clock()-start

r, execTime = check_overlaps(train_dataset, test_dataset)    
print("# overlaps between training and test sets:" + str(len(r)) + " execution time: " + str(execTime))
r, execTime = check_overlaps(train_dataset, valid_dataset)   
print("# overlaps between training and validation sets:"+ str(len(r))+ "execution time:"+str(execTime))
r, execTime = check_overlaps(valid_dataset, test_dataset) 
print("# overlaps between validation and test sets:"+ str(len(r))+ "execution time:"+ str(execTime))

# overlaps between training and test sets:1 execution time: 15.883333320164631
# overlaps between training and validation sets:1execution time:5.3156518468711536
# overlaps between validation and test sets:1execution time:0.6824502795847422


In [28]:
gc.collect()

0